In [2]:
import os
import pandas as pd
'''!pip uninstall numpy
!pip install numpy==1.26.4'''
import numpy as np
from astropy.table import Table
import astropy.units as u
from astropy.io import fits
try:
  from specutils import Spectrum
except:
  !pip install specutils
from astropy.nddata import StdDevUncertainty

# install GLEAM
# 2. Define path to your patched source code
SOURCE_PATH = "/content/drive/MyDrive/Colab_Pip_Packages/gleam_source"

try:
  import gleam
  from numba import jit
  from functools import lru_cache
except:
  !pip install numba

  # 3. Install the package in "editable" mode
  # This creates a link to the code on your Drive instead of copying it.
  print("\n--> Installing your patched version of GLEAM...")
  !pip install -e {SOURCE_PATH}

from numba import jit

# --- Configuration ---
BASE_PATH = "/content/drive/MyDrive/Bren_code/My_work/JAdeS_Analysis/"
SMACS_PROJECT_PATH = os.path.join(BASE_PATH, "GLEAM_JADES/")
MASTER_SMACS_CSV = os.path.join(BASE_PATH, "jades.csv")

In [ ]:
# --- Emission Line Catalog (can be the same as JADES) ---
EMISSION_LINES = {
    # Name: Rest Wavelength (Angstroms)

    # --- Hydrogen: Balmer Series (Visible) ---
    r'H-alpha': 6562.819 * u.AA,
    r'H-beta': 4861.333 * u.AA,
    r'H-gamma': 4340.47 * u.AA,
    r'H-delta': 4101.74 * u.AA,
    r'H-epsilon': 3970.07 * u.AA,

    # --- Hydrogen: Paschen Series (NIR) ---
    r'Pa-alpha': 18750.976 * u.AA,
    r'Pa-beta': 12818.1 * u.AA,
    r'Pa-gamma': 10938.1 * u.AA,
    r'Pa-delta': 10049.0 * u.AA,
    r'Pa-epsilon': 9545.98 * u.AA,
    r'Pa-zeta': 9229.02 * u.AA,
    r'Pa-eta': 9014.91 * u.AA,

    # --- Hydrogen: Brackett Series (NIR) ---
    r'Br-beta': 26251.0 * u.AA,
    r'Br-gamma': 21655.302 * u.AA,
    r'Br-delta': 19445.582 * u.AA,
    r'Br-epsilon': 18174.141 * u.AA,

    # --- Hydrogen: Pfund Series (NIR) ---
    r'Pf-delta': 32960.0 * u.AA,
    r'Pf-epsilon': 30383.731 * u.AA,

    # --- Other notable Hydrogen lines ---
    r'H (15700 A)': 15700.0 * u.AA,
    r'H (26119 A)': 26119.351 * u.AA,
    r'H (28722 A)': 28722.0 * u.AA,

    # --- Helium Lines ---
    r'He I 5875': 5875.0 * u.AA,
    r'He I 10830': 10830.0 * u.AA,
    r'He I 18697': 18697.216 * u.AA,
    r'He II 4686': 4685.68 * u.AA,

    # --- Common Forbidden Lines ("Nebular" Lines) ---
    r'[O I] 6300': 6300 * u.AA,
    r'[O II] 3727': 3727.3 * u.AA,    # Doublet avg.
    r'[O III] 4959': 4958.91 * u.AA,
    r'[O III] 5007': 5006.84 * u.AA,
    r'[N II] 6583': 6583.4 * u.AA,    # Often blended with H-alpha
    r'[S II] 6716': 6716.4 * u.AA,
    r'[S II] 6731': 6730.8 * u.AA,

    # --- High-Ionization / AGN Lines ---
    # These lines indicate a very hard radiation field, often from an AGN.
    # Some are UV lines, only visible in NIR spectra for high-redshift objects.
    r'[C IV] 1549': 1549.0 * u.AA,           # UV, z > ~4 for NIRSpec
    r'[C III] 1909': 1908.7 * u.AA,         # UV, z > ~3 for NIRSpec
    r'[Mg II] 2798': 2798.0 * u.AA,          # UV, z > ~2 for NIRSpec
    r'[Ne V] 3346': 3345.8 * u.AA,
    r'[Ne V] 3426': 3425.9 * u.AA,
    r'[Ne III] 3869': 3868.8 * u.AA,
    r'[Ar IV] 4740': 4740.2 * u.AA,
    r'[Fe VII] 6087': 6087.0 * u.AA,
}

# --- 1. Create the Line List (smacs_lines.fits) ---
print("--> Creating smacs_lines.fits...")
line_table_path = os.path.join(SMACS_PROJECT_PATH, "smacs_lines.fits")
line_names = list(EMISSION_LINES.keys())
wavelengths = u.Quantity([wav.to_value(u.AA) for wav in EMISSION_LINES.values()], unit=u.AA)
line_table = Table([line_names, wavelengths], names=('line', 'wavelength'))
line_table['latex'] = line_table['line']
line_table.write(line_table_path, format='fits', overwrite=True)
print("✓ Success: Created smacs_lines.fits")

# --- 2. Create the Metadata File (meta.smacs.fits) ---
print("\n--> Creating meta.smacs.fits...")
metadata_path = os.path.join(SMACS_PROJECT_PATH, "meta.smacs.fits")
df = pd.read_csv(MASTER_SMACS_CSV)

# --- THIS IS THE CORRECTED SECTION ---
# Helper function to parse the GID from a filename
def extract_gid_from_filename(filename):
    try:
        # Assumes a format like '..._1234.spec.fits' or similar
        # This is the same logic used in your main analysis script
        return int(filename.split('_')[-1].replace('.spec.fits', ''))
    except (ValueError, IndexError):
        return None

# Apply this function to the 'file' column to create a 'galaxy_id' column
df['galaxy_id'] = df['file'].apply(extract_gid_from_filename)

# Drop any rows where we couldn't parse a valid galaxy_id
df.dropna(subset=['galaxy_id'], inplace=True)
df['galaxy_id'] = df['galaxy_id'].astype(int)

# remove any duplicates
df = df.drop_duplicates(subset=['galaxy_id'])

# Rename the 'Redshift' column for consistency
df = df.rename(columns={'Redshift': 'z'})

metadata_table = Table.from_pandas(df)

# Now we can rename 'galaxy_id' because we just created it
metadata_table.rename_column('galaxy_id', 'SourceNumber')
metadata_table.rename_column('z', 'Redshift')
metadata_table['Sample'] = 'SMACS'
metadata_table['Setup'] = 'JWST_NIRSpec'
# Assign a static pointing name since there is no project_id column
metadata_table['Pointing'] = 'SMACS0723'

metadata_table = metadata_table['Sample', 'Setup', 'Pointing', 'SourceNumber', 'Redshift']
metadata_table.write(metadata_path, format='fits', overwrite=True)
print(f"✓ Success: Created meta.smacs.fits with {len(metadata_table)} entries.")
# --- END CORRECTION ---


# --- 3. Create the Configuration File (smacs_config.yaml) ---
print("\n--> Creating smacs_config.yaml...")
config_path = os.path.join(SMACS_PROJECT_PATH, "smacs_config.yaml")
config_content = """
globals:
  line_table: smacs_lines.fits
  mask_sky: False
  fitting:
    SN_limit: 5.0
    cont_width: 70 Angstrom
    center: constrained
setups:
  JWST_NIRSpec:
    resolution: 10 Angstrom
sources: {}
"""
with open(config_path, 'w') as f:
    f.write(config_content.strip())
print("✓ Success: Created smacs_config.yaml")

print("\n--- SMACS Setup Complete! ---")

--> Creating smacs_lines.fits...
✓ Success: Created smacs_lines.fits

--> Creating meta.smacs.fits...
✓ Success: Created meta.smacs.fits with 81 entries.

--> Creating smacs_config.yaml...
✓ Success: Created smacs_config.yaml

--- SMACS Setup Complete! ---


In [ ]:
import os
import numpy as np
import glob
from tqdm.notebook import tqdm
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.wcs import WCS
from specutils import Spectrum1D
from astropy.nddata import StdDevUncertainty

# =========================================
# 1. SETUP
# =========================================
BASE_PATH = "/content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/"
# Input directory with the incorrectly formatted files
INPUT_DIR = os.path.join(BASE_PATH, "GLEAM_SMACS/SMACS/") # Assuming this is where the SMACS files are, as per your ls output
# New output directory for the corrected files
OUTPUT_DIR = os.path.join(BASE_PATH, "GLEAM_SMACS/SMACS/")

os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================================
# 2. HELPER FUNCTIONS
# =========================================

def read_incorrect_spectrum_format(file_path):
    """
    Manually reads the FITS Image format that your script was creating.
    """
    try:
        with fits.open(file_path) as hdul:
            flux_data = hdul[0].data
            flux_unit = u.Unit(hdul[0].header['BUNIT'])
            flux = flux_data * flux_unit

            # Use a default empty uncertainty if the extension is missing
            if 'UNCERT' in hdul:
                uncertainty_data = hdul['UNCERT'].data
                uncertainty_unit = u.Unit(hdul['UNCERT'].header['BUNIT'])
                uncertainty = StdDevUncertainty(uncertainty_data * uncertainty_unit)
            else:
                 uncertainty = StdDevUncertainty(np.zeros_like(flux.value) * flux.unit)

            wcs = WCS(hdul[0].header)
            spectrum = Spectrum1D(flux=flux, wcs=wcs, uncertainty=uncertainty)
            return spectrum
    except Exception as e:
        print(f"--> Could not read {os.path.basename(file_path)}. Reason: {e}")
        return None

def save_correct_spectrum_format(spectrum, output_path, filename):
    """
    Saves a Spectrum1D object to the correct FITS Table format that GLEAM needs.
    """
    valid_mask = np.isfinite(spectrum.flux) & np.isfinite(spectrum.uncertainty.array)
    if not np.any(valid_mask):
        print(f"--> WARNING: Spectrum for {filename} contains no valid data. Skipping save.")
        return

    output_table = Table({
        'wl': spectrum.spectral_axis[valid_mask],
        'flux': spectrum.flux[valid_mask],
        'stdev': spectrum.uncertainty.array[valid_mask] * spectrum.flux.unit
    })
    output_table.write(os.path.join(output_path, filename), format='fits', overwrite=True)

# =========================================
# 3. MAIN CONVERSION LOOP
# =========================================

# Find all the spectrum files in the input directory
files_to_convert = glob.glob(os.path.join(INPUT_DIR, "spec1d.SMACS.*.fits"))

print(f"Found {len(files_to_convert)} spectra to convert...")

for file_path in tqdm(files_to_convert, desc="Converting Files"):
    filename = os.path.basename(file_path)

    # Load the spectrum from the old, incorrect format
    spectrum = read_incorrect_spectrum_format(file_path)

    if spectrum:
        # Save the spectrum in the new, correct format
        save_correct_spectrum_format(spectrum, OUTPUT_DIR, filename)

print("\n--- Conversion Complete! ---")

Found 3 spectra to convert...


Converting Files:   0%|          | 0/3 [00:00<?, ?it/s]


--- Conversion Complete! ---


In [3]:
# ====================================================================
# CELL 1: Install LaTeX Dependencies
# ====================================================================
print("--> Installing full LaTeX dependencies (this may take a minute)...")
!sudo apt-get update > /dev/null
!sudo apt-get install -y texlive-latex-base texlive-fonts-recommended dvipng cm-super texlive-latex-extra texlive-science > /dev/null
print("✅ LaTeX installation complete.")

--> Installing full LaTeX dependencies (this may take a minute)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 60.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
✅ LaTeX installation complete.


In [ ]:
print(f"Running GLEAM with input path: {BASE_PATH}")
!cd /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/ && \
    gleam --path . \
    --config smacs_config.yaml \
    --spectra './SMACS/spec1d.SMACS.*.fits' \
    --plot \
    --nproc 6

Running GLEAM with input path: /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/
--- Matching FITS files to metadata by filename ---
--- Found 4 valid sources to process ---

--- RUNNING IN FORGIVING PARALLEL MODE ---

DEBUG: Globals loaded for source 4590: {'line_list': '/content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/smacs_lines.fits', 'sky_lines': None, 'instrument': {'name': 'JWST_NIRSpec', 'rest_spectral_resolution': '100 km/s'}, 'fitting': {'continuum': True, 'cont_width': '100 Angstrom', 'center_constraint': 'constrained', 'constraints': True, 'mask_sky': False, 'tolerance': '20 Angstrom'}}
DEBUG: Globals loaded for source 9483: {'line_list': '/content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/smacs_lines.fits', 'sky_lines': None, 'instrument': {'name': 'JWST_NIRSpec', 'rest_spectral_resolution': '100 km/s'}, 'fitting': {'continuum': True, 'cont_width': '100 Angstrom', 'center_constraint': 'constrained', 'constraints': True, 'mask_sky'

In [ ]:
import os

# Define the source and destination directories
source_dir = os.path.join(BASE_PATH, "GLEAM_SMACS/SMACS")
destination_dir = os.path.join(BASE_PATH, "GLEAM_plots")

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Use a shell command to move all .png files
!mv {source_dir}/*.png {destination_dir}/

mv: cannot stat '/content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/*.png': No such file or directory


In [ ]:
import os
import glob
from astropy.table import QTable
import pandas as pd

def convert_fits_to_csv(search_path="/content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/"):
    """
    Finds all 'linefits.*.fits' files in a directory and its subdirectories,
    extracts key columns, and saves them to a CSV file named after the source ID.
    """
    # --- FIX: Search recursively to find files in subdirectories ---
    search_pattern = os.path.join(search_path, "**/linefits.*.fits")
    fits_files = glob.glob(search_pattern, recursive=True)

    if not fits_files:
        print("No 'linefits.*.fits' files found to convert.")
        return

    files_by_source = {}
    for f in fits_files:
        try:
            parts = os.path.basename(f).split('.')
            source_id = parts[4]
            if source_id not in files_by_source:
                files_by_source[source_id] = []
            files_by_source[source_id].append(f)
        except IndexError:
            print(f"Warning: Could not parse source ID from filename: {f}")
            continue

    print(f"Found {len(fits_files)} FITS files for {len(files_by_source)} unique sources.")

    for source_id, file_list in files_by_source.items():
        linefits_file = file_list[0]

        try:
            data = QTable.read(linefits_file)

            # --- FIX: Intelligently find the line name column ---
            if 'name' in data.colnames:
                name_col = 'name'
            elif 'line' in data.colnames:
                name_col = 'line'
            else:
                raise KeyError("Could not find a line identifier column (expected 'name' or 'line')")

            required_cols = [name_col, 'center', 'flux', 'flux_err']
            optional_cols = ['fwhm']

            export_cols = required_cols + [col for col in optional_cols if col in data.colnames]

            df = data[export_cols].to_pandas()

            # Rename the column to 'name' for a consistent CSV format
            if name_col != 'name':
                df.rename(columns={name_col: 'name'}, inplace=True)

            # Define the output CSV in the same directory as the FITS file
            output_dir = os.path.dirname(linefits_file)
            csv_filename = os.path.join(output_dir, f"{source_id}.csv")

            df.to_csv(csv_filename, index=False)
            print(f"Successfully converted {linefits_file} to {csv_filename}")

        except Exception as e:
            print(f"Error processing {linefits_file}: {e}")


if __name__ == "__main__":
    convert_fits_to_csv()

Found 4 FITS files for 4 unique sources.
Successfully converted /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/linefits.SMACS.JWST_NIRSpec.SMACS0723.9483.fits to /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/9483.csv
Successfully converted /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/linefits.SMACS.JWST_NIRSpec.SMACS0723.9721.fits to /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/9721.csv
Successfully converted /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/linefits.SMACS.JWST_NIRSpec.SMACS0723.9922.fits to /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/9922.csv
Successfully converted /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/linefits.SMACS.JWST_NIRSpec.SMACS0723.4590.fits to /content/drive/MyDrive/Bren_code/My_work/SMACS_Analysis/GLEAM_SMACS/SMACS/4590.csv
